In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import requests
import json
import io
import os
from dotenv import load_dotenv

### Rainforest Product API

In [3]:
load_dotenv()

api_key = os.getenv('RAINFOREST_API_KEY')

In [ ]:
# set up the request parameters



params = {
    'api_key': api_key,
    'amazon_domain': 'amazon.com',
    'asin': 'B06Y5ZG66K',
    'type': 'product'
}

# make the http GET request to Rainforest API
api_result = requests.get('https://api.rainforestapi.com/request', params)

# print the JSON response from Rainforest API
json_result = json.dumps(api_result.json())

### Rainforest Search API

In [5]:
# set up the request parameters
params = {
    'api_key': api_key,
    'type': 'search',
    'amazon_domain': 'amazon.com',
    'search_term': 'protein powder'
}

# make the http GET request to Rainforest API
api_result = requests.get('https://api.rainforestapi.com/request', params)

# print the JSON response from Rainforest API
search = api_result.json()

In [6]:
search

{'request_info': {'success': True,
  'credits_used': 4,
  'credits_remaining': 96,
  'credits_used_this_request': 1},
 'request_parameters': {'type': 'search',
  'amazon_domain': 'amazon.com',
  'search_term': 'protein powder'},
 'request_metadata': {'created_at': '2024-08-07T03:37:49.507Z',
  'processed_at': '2024-08-07T03:38:00.579Z',
  'total_time_taken': 11.07,
  'amazon_url': 'https://www.amazon.com/s/?k=protein+powder&ref=nb_sb_noss_2'},
 'search_results': [{'position': 1,
   'title': '365 by Whole Foods Market, Protein Whey Grassfed Vanilla, 10.8 Ounce',
   'asin': 'B08F3CC64W',
   'link': 'https://www.amazon.com/sspa/click?ie=UTF8&spc=MTo1OTQ4ODA0MTE0NzEyOTg0OjE3MjMwMDE4NzA6c3BfYXRmOjIwMDA3MjMwMDg4NzQ0MTo6MDo6&url=%2F365-WFM-Vanilla-Protein-Grassfed%2Fdp%2FB08F3CC64W%2Fref%3Dsr_1_1_ffob_sspa%3Fdib%3DeyJ2IjoiMSJ9.OCtFwLzEe-cneMGHIGBBsvaXQg7xmx6SIWl9DG81pdRjLFbJjsw6Q7BPOAmSk9OjuXIY-IIks3PTw0J6lffswUOa-NKYzRvSjROYntw2_fNSMnDAad5_zuN9H4XevAe7KV10_4-EfaC7jTniGU-edeNjHPUojPoJSngnSUvv

In [3]:
# Iterate through search results and store the values we care about

search_values = [
    (r.get('asin', 0), 
     r.get('position', 0),
     r.get('title', 0),
     r.get('link', 0),
     r.get('recent_sales', 0),
     r.get('categories', 0),
     r.get('unit_price', 0),
     r.get('rating', 0),
     r.get('is_prime', 0),
     r.get('ratings_total', 0),
     r.get('sponsored', 0),
     r.get('price', {}).get('value',0),
     r.get('amazons_choice', 0),
     r.get('bestseller', 0),
     r.get('coupon', 0)
    ) for r in search['search_results']]

In [4]:
search_df = pd.DataFrame(search_values, columns = [
    'asin', 
    'position', 
    'title', 
    'link', 
    'recent_sales', 
    'categories', 
    'unit_price', 
    'rating', 
    'is_prime', 
    'ratings_total', 
    'sponsored', 
    'price',
    'amazons_choice', 
    'bestseller', 
    'coupon'])

In [5]:
search_df.head()

,asin,position,title,link,recent_sales,categories,unit_price,rating,is_prime,ratings_total,sponsored,price,amazons_choice,bestseller,coupon
0,B07VPKSD15,1,"GHOST Whey Protein Powder, Chips Ahoy - 2LB Tu...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,10K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.28$1.28/Ounce,4.5,True,11167,True,44.99,0,0,0
1,B0CL8QYFYF,2,"FlavCity Protein Powder Smoothie, Butter Coffe...",https://www.amazon.com/sspa/click?ie=UTF8&spc=...,5K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.81$1.81/Ounce,4.2,0,3064,True,59.99,0,0,"{'badge_text': 'Save 5%', 'text': 'with coupon'}"
2,B0D2PDNT55,3,NAKED Whey Chocolate Protein Powder - Only 3 I...,https://www.amazon.com/sspa/click?ie=UTF8&spc=...,2K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.31$1.31/Ounce,4.0,True,16880,True,41.99,0,0,0
3,B072HS77TG,4,Ka'Chava Shakes - Vanilla (Vanilla),https://www.amazon.com/sspa/click?ie=UTF8&spc=...,5K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$2.52$2.52/Ounce,4.3,0,9212,True,79.99,0,0,0
4,B002DYIZH6,5,Optimum Nutrition Gold Standard 100% Whey Prot...,https://www.amazon.com/Optimum-Nutrition-Delic...,40K+ bought in past month,"[{'name': 'All Departments', 'id': 'aps'}]",$1.10$1.10/Ounce,4.6,True,211072,0,35.29,0,{'link': 'https://www.amazon.com/gp/bestseller...,"{'badge_text': 'Save 15%', 'text': 'with coupon'}"
